<a href="https://colab.research.google.com/github/abhigyan2003/NLP/blob/main/OMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import threading
import time
from collections import deque
from datetime import datetime


THROTTLE_LIMIT = 100
ACTIVE_HOURS = ("09:00", "17:00")


class RequestType:
    Unknown = 0
    New = 1
    Modify = 2
    Cancel = 3

class ResponseType:
    Unknown = 0
    Accept = 1
    Reject = 2


class OrderRequest:
    def __init__(self, symbolId, price, qty, side, orderId, requestType):
        self.m_symbolId = symbolId
        self.m_price = price
        self.m_qty = qty
        self.m_side = side
        self.m_orderId = orderId
        self.m_requestType = requestType

class OrderResponse:
    def __init__(self, orderId, responseType):
        self.m_orderId = orderId
        self.m_responseType = responseType


class OrderManagement:
    def __init__(self):
        self.queue = deque()
        self.sent_orders = {}
        self.order_metadata = {}
        self.lock = threading.Lock()
        self.last_sent_second = None
        self.orders_sent_this_second = 0
        self.logged_on = False

        self._start_time_monitor()
        self._start_queue_processor()

    def _get_current_ist_time(self):
        return datetime.now()

    def _is_within_trading_window(self):
        now = self._get_current_ist_time().time()
        start_time = datetime.strptime(ACTIVE_HOURS[0], "%H:%M").time()
        end_time = datetime.strptime(ACTIVE_HOURS[1], "%H:%M").time()
        return start_time <= now <= end_time

    def _start_time_monitor(self):
        def monitor():
            while True:
                with self.lock:
                    if self._is_within_trading_window() and not self.logged_on:
                        self.sendLogon()
                        self.logged_on = True
                    elif not self._is_within_trading_window() and self.logged_on:
                        self.sendLogout()
                        self.logged_on = False
                time.sleep(5)
        threading.Thread(target=monitor, daemon=True).start()

    def _start_queue_processor(self):
        def process():
            while True:
                with self.lock:
                    self._process_queue()
                time.sleep(1)
        threading.Thread(target=process, daemon=True).start()

    def _throttle_send(self):
        now = datetime.now()
        if self.last_sent_second != now.second:
            self.last_sent_second = now.second
            self.orders_sent_this_second = 0
        return self.orders_sent_this_second < THROTTLE_LIMIT

    def _process_queue(self):
        while self.queue and self._throttle_send():
            order = self.queue.popleft()
            self._send(order)

    def onData(self, request: OrderRequest):
        with self.lock:
            if not self._is_within_trading_window():
                print(f"Order {request.m_orderId} REJECTED: Outside trading window")
                return

            if request.m_requestType == RequestType.New:
                if self._throttle_send():
                    self._send(request)
                else:
                    self.queue.append(request)

            elif request.m_requestType == RequestType.Modify:
                for queued_order in self.queue:
                    if queued_order.m_orderId == request.m_orderId:
                        queued_order.m_price = request.m_price
                        queued_order.m_qty = request.m_qty
                        print(f"Order {request.m_orderId} MODIFIED in queue")
                        break

            elif request.m_requestType == RequestType.Cancel:
                old_len = len(self.queue)
                self.queue = deque([o for o in self.queue if o.m_orderId != request.m_orderId])
                if len(self.queue) < old_len:
                    print(f"Order {request.m_orderId} CANCELED from queue")

    def _send(self, request: OrderRequest):
        print(f"Sending order {request.m_orderId}")
        self.orders_sent_this_second += 1
        self.sent_orders[request.m_orderId] = request
        self.order_metadata[request.m_orderId] = datetime.now()

    def onDataResponse(self, response: OrderResponse):
        with self.lock:
            sent_time = self.order_metadata.pop(response.m_orderId, None)
            if sent_time:
                latency = (datetime.now() - sent_time).total_seconds() * 1000  # in ms
                self._log_latency(response, latency)
            else:
                print(f"Response received for unknown order {response.m_orderId}")

    def _log_latency(self, response: OrderResponse, latency: float):
        with open("latency_log.txt", "a") as f:
            f.write(f"OrderID: {response.m_orderId}, Response: {response.m_responseType}, Latency: {latency:.2f}ms\n")
        print(f"Logged latency for Order {response.m_orderId}: {latency:.2f} ms")

    def sendLogon(self):
        print("Logon message sent to exchange")

    def sendLogout(self):
        print(" Logout message sent to exchange")




#i used this for just testing the model

if __name__ == "__main__":
    oms = OrderManagement()


    for i in range(105):
        req = OrderRequest(symbolId=1, price=100+i, qty=10+i, side='B', orderId=i, requestType=RequestType.New)
        oms.onData(req)


    modify_req = OrderRequest(symbolId=1, price=999.99, qty=99, side='B', orderId=102, requestType=RequestType.Modify)
    cancel_req = OrderRequest(symbolId=1, price=0, qty=0, side='B', orderId=104, requestType=RequestType.Cancel)
    oms.onData(modify_req)
    oms.onData(cancel_req)


    time.sleep(2)


    for i in range(105):
        response = OrderResponse(orderId=i, responseType=ResponseType.Accept)
        oms.onDataResponse(response)

    print("\nCheck latency_log.txt for logs.")


Order 0 REJECTED: Outside trading window
Order 1 REJECTED: Outside trading window
Order 2 REJECTED: Outside trading window
Order 3 REJECTED: Outside trading window
Order 4 REJECTED: Outside trading window
Order 5 REJECTED: Outside trading window
Order 6 REJECTED: Outside trading window
Order 7 REJECTED: Outside trading window
Order 8 REJECTED: Outside trading window
Order 9 REJECTED: Outside trading window
Order 10 REJECTED: Outside trading window
Order 11 REJECTED: Outside trading window
Order 12 REJECTED: Outside trading window
Order 13 REJECTED: Outside trading window
Order 14 REJECTED: Outside trading window
Order 15 REJECTED: Outside trading window
Order 16 REJECTED: Outside trading window
Order 17 REJECTED: Outside trading window
Order 18 REJECTED: Outside trading window
Order 19 REJECTED: Outside trading window
Order 20 REJECTED: Outside trading window
Order 21 REJECTED: Outside trading window
Order 22 REJECTED: Outside trading window
Order 23 REJECTED: Outside trading window
Or